# Captum을 사용한 모델 이해
아래 동영상을 시청하거나 youtube 링크를 확인하세요. 노트북 및 관련 파일은 여기에서 다운로드할 수 있습니다.

<div style="margin-top:10px; margin-bottom:10px;">
  <iframe width="560" height="315" src="https://www.youtube.com/embed/Am2EF9CLu-g" frameborder="0" allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</div>

Captum ("comprehension" 라틴어)은 PyTorch 기반의 오픈 소스이며, 모델 해석을 위한 확장 가능한 라이브러리입니다.

모델 복잡성의 증가로 투명성이 감소함에 따라 모델 해석 방법이 점점 더 중요해지고 있습니다. 모델 이해는 연구의 활발한 분야이며, 머신 러닝을 사용하는 다양한 산업에서의 실제 응용에 중점을 둔 분야입니다. Captum은 통합된 그라디언트를 포함한 최첨단 알고리즘을 제공하여 연구자와 개발자가 모델 출력에 기여하는 특징을 쉽게 이해할 수 있도록 도와줍니다.

Captum.ai 웹사이트에서 전체 문서, API 참조, 특정 주제에 대한 튜토리얼 세트를 사용할 수 있습니다.

## 소개
Captum의 모델 해석 방법은 속성을 기준으로 합니다. Captum에서 사용 가능한 세 가지 종류의 속성이 있습니다.

피처 속성: 생성된 입력의 특징을 기반으로 한 출력을 설명하려고 노력합니다. 리뷰에서 특정 단어의 유무에 따라 영화 리뷰가 긍정적인지 부정적인지 설명하는 것은 피처 속성의 예입니다.
레이어 속성: 특정 입력 후에 모델의 숨겨진 레이어 활동을 조사합니다. 입력 이미지에 대한 컨볼루션 레이어의 공간적으로 매핑된 출력을 조사하는 것이 레이어 속성의 예입니다.
뉴런 속성: 레이어 속성과 유사하지만 하나의 뉴런의 활동에 중점을 둡니다.
이 상호작용적인 노트북에서는 피처 속성 및 레이어 속성을 살펴볼 것입니다.

세 가지 속성 유형 각각에는 여러 속성 알고리즘이 연결되어 있습니다. 많은 속성 알고리즘이 두 가지 넓은 범주로 나뉩니다.

그라디언트 기반 알고리즘: 모델 출력, 레이어 출력 또는 뉴런 활성화에 대한 입력에 대한 역방향 그라디언트를 계산합니다. 통합 그라디언트 (피처용), 레이어 그라디언트 * 활성화 및 뉴런 컨덕턴스는 모두 그라디언트 기반 알고리즘입니다.
왜곡 기반 알고리즘: 모델, 레이어 또는 뉴런의 출력 변경을 조사합니다. 입력의 변경에 대한 모델, 레이어 또는 뉴런의 출력 변화를 조사합니다. 입력 왜곡은 지정된 또는 무작위일 수 있습니다. 가려짐, 피처 삭제 및 피처 순열은 모두 왜곡 기반 알고리즘입니다.
이 노트북에서는 두 유형의 알고리즘을 모두 살펴볼 것입니다.

특히 대규모 모델이 관련된 경우 속성 데이터를 입력 특징과 쉽게 관련 짓기 위해 시각적으로 표시하는 것이 가치 있습니다. Matplotlib, Plotly 또는 유사한 도구로 직접 시각화를 작성할 수 있지만, Captum은 해당 속성에 특화된 향상된 도구를 제공합니다.

captum.attr.visualization 모듈(아래에서 viz로 가져옴)은 이미지와 관련된 속성에 대한 시각적 도구를 제공합니다.
Captum Insights는 Captum 상단의 사용하기 쉬운 API로 이미지, 텍스트 및 임의의 모델 유형에 대한 시각적 표현을 제공하는 시각화 위젯을 제공합니다.
이 두 가지 시각화 도구세트는 이 노트북에서 설명될 것입니다. 처음 몇 가지 예제는 컴퓨터 비전 사용 사례에 중점을 둘 것이지만, 마지막에는 Captum Insights 섹션에서 다중 모델, 시각적 질문 응답 모델에서의 속성 시각화를 보여줄 것입니다.

### 설치
시작하기 전에 Python 환경이 필요합니다.

Python 버전 3.6 이상
Captum Insights 예제를 위해 Flask 1.1 이상
PyTorch 버전 1.2 이상 (최신 버전 권장)
TorchVision 버전 0.6 이상 (최신 버전 권장)
Captum (최신 버전 권장)
Anaconda 또는 pip 가상 환경에서 Captum을 설치하려면 아래 환경에 맞는 명령어를 사용하세요.

conda를 사용하는 경우::

```conda install pytorch torchvision captum -c pytorch```

pip를 사용하는 경우::

```pip install torch torchvision captum```
노트북을 설정한 환경에서 이 노트북을 다시 시작하면 됩니다.

## 첫 번째 예제
먼저, 간단한 시각적인 예제로 시작하겠습니다. 우리는 ImageNet 데이터셋에 사전 훈련된 ResNet 모델을 사용할 것입니다. 테스트 입력을 가져오고 다양한 피처 속성 알고리즘을 사용하여 입력 이미지가 출력에 어떤 영향을 미치는지를 조사하고 몇 가지 테스트 이미지에 대한 이 입력 속성 맵을 시각적으로 확인해보겠습니다.

먼저, 몇 가지 가져오기를 하겠습니다:

In [7]:
pip install torch torchvision captum

In [8]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models

import captum
from captum.attr import IntegratedGradients, Occlusion, LayerGradCam, LayerAttribution
from captum.attr import visualization as viz

import os, sys
import json

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap


이제 TorchVision 모델 라이브러리를 사용하여 사전 훈련된 ResNet을 다운로드할 것입니다. 훈련을 수행하지 않으므로 현재 평가 모드로 설정하겠습니다.

In [9]:
model = models.resnet101(weights='IMAGENET1K_V1')
model = model.eval()

이 상호작용 노트북을 얻은 위치에는 img 폴더가 있어야 하며 해당 폴더에 cat.jpg 파일이 있어야 합니다.

In [ ]:
test_img = Image.open('img/cat.jpg')
test_img_data = np.asarray(test_img)
plt.imshow(test_img_data)
plt.show()

우리의 ResNet 모델은 ImageNet 데이터셋에서 훈련되었으며 이미지가 특정 크기여야 하며 채널 데이터가 특정 값 범위로 정규화되어야 합니다. 또한 모델이 인식하는 카테고리에 대한 사람이 읽을 수 있는 레이블 목록을 가져올 것입니다. 이는 img 폴더에 있어야 합니다.

In [ ]:
# model expects 224x224 3-color image
transform = transforms.Compose([
 transforms.Resize(224),
 transforms.CenterCrop(224),
 transforms.ToTensor()
])

# standard ImageNet normalization
transform_normalize = transforms.Normalize(
     mean=[0.485, 0.456, 0.406],
     std=[0.229, 0.224, 0.225]
 )

transformed_img = transform(test_img)
input_img = transform_normalize(transformed_img)
input_img = input_img.unsqueeze(0) # the model requires a dummy batch dimension

labels_path = 'img/imagenet_class_index.json'
with open(labels_path) as json_data:
    idx_to_labels = json.load(json_data)


이제 우리는 이 이미지가 모델에 의해 어떤 대상으로 간주되는지에 대한 질문을 할 수 있습니다.

In [ ]:
output = model(input_img)
output = F.softmax(output, dim=1)
prediction_score, pred_label_idx = torch.topk(output, 1)
pred_label_idx.squeeze_()
predicted_label = idx_to_labels[str(pred_label_idx.item())][1]
print('Predicted:', predicted_label, '(', prediction_score.squeeze().item(), ')')

우리는 ResNet이 이 고양이 이미지를 실제로 고양이로 간주한다는 것을 확인했습니다. 그러나 모델이 왜 이 이미지를 고양이로 간주하는지에 대한 답을 얻기 위해 Captum에게 물어봅니다.

통합 그라디언트를 사용한 피처 속성
피처 속성은 특정 출력을 입력의 특징에 할당합니다. 여기서는 특정 입력(즉, 테스트 이미지)을 사용하여 각 입력 피처가 특정 출력 피처에 대한 상대적 중요도를 나타내는 맵을 생성합니다.

Integrated Gradients는 Captum에서 제공하는 피처 속성 알고리즘 중 하나입니다. Integrated Gradients는 모델 출력의 입력에 대한 그라디언트의 적분을 근사화하여 각 입력 피처에 중요도 점수를 할당합니다.

우리의 경우, 우리는 출력 벡터의 특정 요소를 취할 것입니다. 즉, 모델이 선택한 범주에 대한 자신감을 나타내는 요소를 사용하고 Integrated Gradients를 사용하여 이 출력에 기여한 입력 이미지의 어떤 부분을 이해합니다.

Integrated Gradients에서 얻은 중요도 맵이 있으면 Captum의 시각화 도구를 사용하여 중요도 맵을 유용하게 표시합니다. Captum의 visualize_image_attr() 함수는 속성 데이터의 표시를 사용자 정의하는 여러 옵션을 제공합니다. 여기에서는 사용자 정의 Matplotlib 컬러 맵을 전달합니다.

일반적으로 integrated_gradients.attribute() 호출이 포함된 셀을 실행하는 데는 1~2분 정도 소요됩니다.

In [ ]:
# Initialize the attribution algorithm with the model
integrated_gradients = IntegratedGradients(model)

# Ask the algorithm to attribute our output target to
attributions_ig = integrated_gradients.attribute(input_img, target=pred_label_idx, n_steps=200)

# Show the original image for comparison
_ = viz.visualize_image_attr(None, np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1,2,0)),
                      method="original_image", title="Original Image")

default_cmap = LinearSegmentedColormap.from_list('custom blue',
                                                 [(0, '#ffffff'),
                                                  (0.25, '#0000ff'),
                                                  (1, '#0000ff')], N=256)

_ = viz.visualize_image_attr(np.transpose(attributions_ig.squeeze().cpu().detach().numpy(), (1,2,0)),
                             np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1,2,0)),
                             method='heat_map',
                             cmap=default_cmap,
                             show_colorbar=True,
                             sign='positive',
                             title='Integrated Gradients')

위의 이미지에서는 Integrated Gradients가 이미지에서 고양이의 위치 주변에서 가장 강한 신호를 제공한다는 것을 확인할 수 있어야 합니다.

## Occlusion을 사용한 피처 속성
그라디언트 기반의 속성 방법은 모델이 입력에 대한 출력 변화를 직접 계산하여 이해하는 데 도움을 줍니다. 퍼터베이션 기반의 속성 방법은 입력에 변화를 도입하여 출력에 미치는 효과를 측정함으로써 이를 더 직접적으로 처리합니다. Occlusion은 이러한 방법 중 하나입니다. 이는 입력 이미지의 일부를 대체하고 출력 신호에 미치는 효과를 조사하는 것을 포함합니다.

아래에서는 Occlusion 속성을 설정합니다. 합성곱 신경망을 구성하는 것과 유사하게 대상 영역의 크기와 개별 측정 간의 간격을 결정하는 스트라이드 길이를 지정할 수 있습니다. Occlusion 속성의 출력을 visualize_image_attr_multiple()를 사용하여 시각화합니다. 이는 영역별로 양수와 음수 속성의 열 지도를 보여주며, 원본 이미지를 양수 속성 영역으로 마스킹합니다. 이 마스킹은 모델이 가장 "고양이와 비슷한" 것으로 판단한 이미지의 어떤 영역인지를 매우 명확하게 보여줍니다.

In [ ]:
occlusion = Occlusion(model)

attributions_occ = occlusion.attribute(input_img,
                                       target=pred_label_idx,
                                       strides=(3, 8, 8),
                                       sliding_window_shapes=(3,15, 15),
                                       baselines=0)


_ = viz.visualize_image_attr_multiple(np.transpose(attributions_occ.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      ["original_image", "heat_map", "heat_map", "masked_image"],
                                      ["all", "positive", "negative", "positive"],
                                      show_colorbar=True,
                                      titles=["Original", "Positive Attribution", "Negative Attribution", "Masked"],
                                      fig_size=(18, 6)
                                     )


다시 한번 고양이를 포함하는 이미지 영역에 더 큰 중요성이 부여되었음을 확인할 수 있습니다.

Layer GradCAM을 사용한 레이어 속성
레이어 속성을 사용하면 모델의 숨겨진 레이어의 활동을 입력의 특징에 할당할 수 있습니다. 아래에서는 모델 내의 합성곱 레이어 중 하나의 활동을 조사하기 위해 레이어 속성 알고리즘을 사용합니다.

GradCAM은 목표 출력에 대한 주어진 레이어의 그라디언트를 계산하고 각 출력 채널(출력의 차원 2)에 대해 평균을 내고 각 채널에 대한 평균 그라디언트를 레이어 활성화와 곱합니다. 결과는 모든 채널에 대해 합산됩니다. GradCAM은 합성곱 신경망을 위해 설계되었으며, 합성곱 레이어의 활동이 입력과 공간적으로 매핑되기 때문에 GradCAM 속성은 종종 업샘플링되어 입력을 마스킹하는 데 사용됩니다.

레이어 속성은 입력 속성과 유사하게 설정되지만, 모델뿐만 아니라 조사하려는 모델 내의 숨겨진 레이어를 지정해야 합니다. 위와 마찬가지로 attribute()를 호출할 때 관심 있는 대상 클래스를 지정합니다.

In [ ]:
layer_gradcam = LayerGradCam(model, model.layer3[1].conv2)
attributions_lgc = layer_gradcam.attribute(input_img, target=pred_label_idx)

_ = viz.visualize_image_attr(attributions_lgc[0].cpu().permute(1,2,0).detach().numpy(),
                             sign="all",
                             title="Layer 3 Block 1 Conv 2")


우리는 입력 이미지와 비교하기 위해 레이어 속성 데이터를 업샘플링하기 위해 LayerAttribution_ 베이스 클래스의 편리한 메소드인 interpolate()를 사용할 것입니다.

In [ ]:
upsamp_attr_lgc = LayerAttribution.interpolate(attributions_lgc, input_img.shape[2:])

print(attributions_lgc.shape)
print(upsamp_attr_lgc.shape)
print(input_img.shape)

_ = viz.visualize_image_attr_multiple(upsamp_attr_lgc[0].cpu().permute(1,2,0).detach().numpy(),
                                      transformed_img.permute(1,2,0).numpy(),
                                      ["original_image","blended_heat_map","masked_image"],
                                      ["all","positive","positive"],
                                      show_colorbar=True,
                                      titles=["Original", "Positive Attribution", "Masked"],
                                      fig_size=(18, 6))

이러한 시각화는 숨겨진 레이어가 입력에 어떻게 응답하는지에 대한 새로운 통찰력을 얻을 수 있습니다.

Captum Insights는 Captum 위에 구축된 해석성 시각화 위젯으로, 모델 이해를 용이하게 하는 데 도움을 줍니다. Captum Insights는 이미지, 텍스트 및 기타 기능에 걸쳐 기능 속성을 이해하는 데 도움을 주며 여러 입력/출력 쌍에 대한 속성을 시각화하고 이미지, 텍스트 및 임의의 데이터에 대한 시각화 도구를 제공합니다.

이 노트북의 이 부분에서는 Captum Insights를 사용하여 여러 이미지 분류 추론을 시각화할 것입니다.

먼저 몇 장의 이미지를 모아보고 모델이 이러한 이미지에 대해 어떻게 생각하는지 확인해봅시다. 다양성을 위해 고양이, 주전자 및 삼엽충 화석을 가져오겠습니다.







In [ ]:
imgs = ['img/cat.jpg', 'img/teapot.jpg', 'img/trilobite.jpg']

for img in imgs:
    img = Image.open(img)
    transformed_img = transform(img)
    input_img = transform_normalize(transformed_img)
    input_img = input_img.unsqueeze(0) # the model requires a dummy batch dimension

    output = model(input_img)
    output = F.softmax(output, dim=1)
    prediction_score, pred_label_idx = torch.topk(output, 1)
    pred_label_idx.squeeze_()
    predicted_label = idx_to_labels[str(pred_label_idx.item())][1]
    print('Predicted:', predicted_label, '/', pred_label_idx.item(), ' (', prediction_score.squeeze().item(), ')')

… 그리고 우리의 모델은 모두 올바르게 식별하는 것으로 보입니다. 그러나 물론 더 깊게 파고들고 싶습니다. 이를 위해 아래에서 가져온 AttributionVisualizer 객체로 구성된 Captum Insights 위젯을 사용합니다. AttributionVisualizer는 데이터 배치를 기대하므로 Captum의 Batch 도우미 클래스를 가져올 것입니다. 또한 이미지를 살펴볼 것이므로 ImageFeature도 가져올 것입니다.

우리는 AttributionVisualizer를 다음과 같은 인수로 구성합니다:

- 조사할 모델의 배열 (우리의 경우 하나뿐)
- 스코어링 함수, Captum Insights가 모델에서 상위 k개 예측을 가져올 수 있게 함
- 모델이 훈련된 클래스의 정렬된 사람이 읽을 수 있는 목록
- 찾을 피처의 목록 - 우리의 경우 ImageFeature
- 훈련에 사용할 것처럼 배치별 입력 및 레이블을 반환하는 이터러블 객체인 데이터셋

In [ ]:
from captum.insights import AttributionVisualizer, Batch
from captum.insights.attr_vis.features import ImageFeature

# Baseline is all-zeros input - this may differ depending on your data
def baseline_func(input):
    return input * 0

# merging our image transforms from above
def full_img_transform(input):
    i = Image.open(input)
    i = transform(i)
    i = transform_normalize(i)
    i = i.unsqueeze(0)
    return i


input_imgs = torch.cat(list(map(lambda i: full_img_transform(i), imgs)), 0)

visualizer = AttributionVisualizer(
    models=[model],
    score_func=lambda o: torch.nn.functional.softmax(o, 1),
    classes=list(map(lambda k: idx_to_labels[k][1], idx_to_labels.keys())),
    features=[
        ImageFeature(
            "Photo",
            baseline_transforms=[baseline_func],
            input_transforms=[],
        )
    ],
    dataset=[Batch(input_imgs, labels=[282,849,69])]
)

위의 셀을 실행하는 데는 위의 속성과는 달리 많은 시간이 걸리지 않았습니다. 이는 Captum Insights가 시각적 위젯에서 다양한 속성 알고리즘을 구성한 다음 속성을 계산하고 표시하기 때문입니다. 이 프로세스는 몇 분 정도 소요됩니다.

아래의 셀을 실행하면 Captum Insights 위젯이 렌더링됩니다. 그런 다음 속성 방법과 해당 인수를 선택하고 예측된 클래스 또는 예측 정확도를 기반으로 모델 응답을 필터링하고 모델의 예측과 관련된 확률과 함께 모델 응답을 볼 수 있으며 원본 이미지와 비교한 속성의 열 지도를 볼 수 있습니다.

In [ ]:
visualizer.render()